<style>
.text_cell_render {
font-family: Times New Roman, serif;
}
</style>

**Max Marks:** 30
<br>
**Due Date:** Feb 17 2021, 11:59 PM

# General Instructions
- Do not plagiarize. Institute plagiarism policy will be strictly followed.
- Code implementation should be **generic** and should not contain hardcoded values.
- Kindly **document** the code (evaluation might be done **offline** i.e., demos may not happen - hence encouraged).
- Remember to **turn in** after uploading on classroom.
- Your submission should be a single notebook file, **rollNo\_A1.ipynb** (rename this notebook and submit accordingly).

# Objective
- To code up generic expressions of (conditional)probability distributions with the help of below example

# Problem Statement

<div align="center">
    <figure>
      <img src="https://docs.google.com/uc?export=download&id=1ZyH5dqB7xGJLQdQdfIGllRK3pbCHnKGp">
      <figcaption>Figure 1: Bayes Net</figcaption>
    </figure>
</div>

Fig 1. shows a bayesian network over (_A, B, C, D, E_) with the following CPTs:

<pre>

P(B):                                                P(C):
+----------+----------+                              +----------+----------+
|    B=0   |   0.001  |                              |    C=0   |   0.002  |
+----------+----------+                              +----------+----------+
|    B=1   |   0.999  |                              |    C=1   |   0.998  |
+----------+----------+                              +----------+----------+

P(D|A):                                              P(E|A):
+----------+----------+-----------+                  +----------+----------+-----------+
|          |   D=0    |    D=1    |                  |          |   E=0    |    E=1    |
+----------+----------+-----------+                  +----------+----------+-----------+
|    A=0   |   0.95   |    0.05   |                  |    A=0   |   0.99   |    0.01   |
+----------+----------+-----------+                  +----------+----------+-----------+
|    A=1   |   0.10   |    0.90   |                  |    A=1   |   0.30   |    0.70   |
+----------+----------+-----------+                  +----------+----------+-----------+

P(A | B, C):
+-----------+------------+-----------+
|           |    A=0     |    A=1    |
+-----------+------------+-----------+
|  B=0, C=0 |    0.05    |    0.95   | 
+-----------+------------+-----------+
|  B=0, C=1 |    0.71    |    0.29   |
+-----------+------------+-----------+
|  B=1, C=0 |    0.06    |    0.94   |
+-----------+------------+-----------+
|  B=1, C=1 |    0.999   |    0.001  |
+-----------+------------+-----------+

</pre>

Implement a **program** that computes and prints out the probability of any combination of events given any other combination of events.
<br>
**Few example queries:** (_your code should support this UI_)
- A0 | E1  $\hspace{1.7 cm}\equiv\hspace{0.5 cm}$  P(A=0 | E=1)
- C0 | A0, B1  $\hspace{1.05 cm}\equiv\hspace{0.5 cm}$  P(C=0 | A=0, B=1)
- B1 A0 C0 D1 E1 $\hspace{0.25 cm}\equiv\hspace{0.5 cm}$ P(B=1, A=0, C=0, D=1, E=1)

# Your Code

### 1. Importing Libaries

In [15]:
import numpy as np
import itertools
from itertools import product  # For ordered pairs
import pandas as pd
from functools import reduce

### 2. Model Buliding Function

In [16]:
# This function is used to get the model paramters input (Generic Case)

def GetModel():
  
  print('Enter Network Data : ')
  n = int(input('Input number of variables : '))

  # Names of variables
  name_bay = []

  # Cardinality of variables
  card_bay = []

  # Numbr of parents
  parent_bay = []

  # Parents set
  parent = []

  # Total parameters at each nodes
  total_node_parameters = []

  # Store cpds of nodes
  cpds = []
  rv_perm = []

  for i in range(n):
    print('\nInput information about variable ',i+1)
    name = input('\tInput Variable Name : ')
    card = int(input('\tInput cardinality of variable : '))
    nop = int(input('\tEnter the number of parents: '))
    par = []
    if nop != 0 :
      par = list(input('\tEnter the parents(seperated by space) : ').strip().split(' '))
      #par.sort()  
    
    name_bay.append(name)
    card_bay.append(card)
    parent_bay.append(nop)
    parent.append(par)
    
  # Calculating total parameters for each cpd table at each node
  for i in range(n):
    f = 1
    x = parent[i]
    t1 = []
    for j in range(len(x)):
      f = f*card_bay[name_bay.index(x[j])]
      t1.append(card_bay[name_bay.index(x[j])])
    
    total_node_parameters.append(f)  
    rv_perm.append(t1)

  print('\nInput CPD data : ')
  for i in range(n):
    node_cpd = []
    t2 = []

    print('\n\tEnter CPD values for Node ',name_bay[i])
    node_cpd = list(map(float,input('\tTotal Values to enter : '+str(card_bay[i]*total_node_parameters[i])+' (seperated by space) ==> ').strip().split(' ')))
    
    # Arranging the input data into tabular form
    if parent_bay[i] ==0:  
      node_cpd_df = pd.DataFrame([m for m in range(card_bay[i])],columns = [name_bay[i]])
    else:  
      for k in range(parent_bay[i]):
        t2.append([l for l in range(rv_perm[i][k])])
      
      t2.append([l for l in range(card_bay[i])])  
      
      # Generating combinations of ordered pairs
      t3 = list(itertools.product(*t2))
      t4 = [list(t) for t in t3]
      col = parent[i]+[name_bay[i]]
      t4.sort(key = lambda x: x[len(col)-1]) 
      node_cpd_df = pd.DataFrame(t4,columns = col)
    
    # Appending the cpd data into the dataframe
    if len(parent[i])==0:
      prob = 'P('+name_bay[i]+')'
    else: 
      t5 = ','.join(parent[i]) 
      prob = 'P('+name_bay[i]+'/'+t5+')'

    node_cpd_df[prob] = node_cpd
    print(node_cpd_df)
    cpds.append(node_cpd_df)
  
  return  n,name_bay,card_bay,parent_bay,parent,total_node_parameters,cpds,rv_perm



In [17]:
# Merging CPD tables 

def Merge_CPD(cpds):

  # Creating copy of cpds list
  cpds_c = cpds.copy()

  # Assigning merged_cpds as single cpd at index 0
  merged_cpds = cpds[0]
  count = 1

  # To keep count of the indices of cpds that have been used in the merging process
  done = []

  # Loop to merge all the cpds until we get single cpd
  while(count<=len(cpds_c)):
    for i in range(1,len(cpds_c)):
      if i in done:
            continue
      else:

        # Finding the common column between the merged dataframe and the ith index cpd dataframe 
        comm = set.intersection(set(merged_cpds.columns),set(cpds_c[i].columns))
        if len(comm)>0:
          merged_cpds = pd.merge(merged_cpds,cpds_c[i],on =list(comm))
          done.append(i)
          break  

    count  = count + 1

  # Arranging the merged dataframe  
  mcol = list(merged_cpds.columns)
  mcol.sort()
  if len(list(mcol[0]))==1:
    merged_cpds = merged_cpds[mcol]
  else :
    merged_cpds = merged_cpds[mcol[::-1]]
  merge_c = merged_cpds.copy()  

  return merged_cpds,merge_c


### 3. Inference Generating Function

In [18]:
# This function will get the query input and the merged CPD tables
def GetAnswer(inp,merge_c):

  # Query processing to get the parameters from query
  query = inp.split(', ')
  query = ' '.join(query)
  query = query.split(' ')
  
  t1 = []
  t2 = []
  t3 = []
  t4 = []
  t5 = []
  while '' in query:
    query.remove('')

  # Inferring from the merged CPD tables
  if query.count('|') ==1:
    i = 0
    while(query[i]!='|'):

      # Generating the query part
      t1.append(list(query[i])[0])
      t2.append(int(list(query[i])[1]))
      i = i+1
    t3 = query[query.index('|')+1:]

    # Generating the evidence part
    for j in range(len(t3)):
      t4.append(list(t3[j])[0])
      t5.append(int(list(t3[j])[1]))

    t1 = t1 + t4
    t2 = t2 + t5  

    m1 = merge_c.copy()
    m2 = merge_c.copy()
    
    for k in range(len(t1)):
        m1 = m1[m1[t1[k]] == t2[k]]
    
    r1 = np.array(m1.iloc[:,n:])
    r1 = np.prod(r1,axis=1)
    r1 = np.sum(r1)

    for k in range(len(t4)):
      m2 = m2[m2[t4[k]] == t5[k]]

    r2 = np.array(m2.iloc[:,n:])
    r2 = np.prod(r2,axis=1)
    r2 = np.sum(r2)

    result = r1/r2
    
  else :
    for k in range(len(query)):
      t1.append(list(query[k])[0])
      t2.append(int(list(query[k])[1]))
      for i in range(len(t1)):
        merge_c = merge_c[merge_c[t1[i]] == t2[i]]
        
    result = np.array(merge_c.iloc[:,n:])
    result = np.prod(result,axis=1)
    result = np.sum(result)

  return result  


### Examples

#### Example 1. (Above Given Network)


In [22]:
# Generate Network 
n,name_bay,card_bay,parent_bay,parent,total_node_parameters,cpds,rv_perm = GetModel()


Enter Network Data : 
Input number of variables : 5

Input information about variable  1
	Input Variable Name : B
	Input cardinality of variable : 2
	Enter the number of parents: 0

Input information about variable  2
	Input Variable Name : C
	Input cardinality of variable : 2
	Enter the number of parents: 0

Input information about variable  3
	Input Variable Name : A
	Input cardinality of variable : 2
	Enter the number of parents: 2
	Enter the parents(seperated by space) : B C

Input information about variable  4
	Input Variable Name : D
	Input cardinality of variable : 2
	Enter the number of parents: 1
	Enter the parents(seperated by space) : A

Input information about variable  5
	Input Variable Name : E
	Input cardinality of variable : 2
	Enter the number of parents: 1
	Enter the parents(seperated by space) : A

Input CPD data : 

	Enter CPD values for Node  B
	Total Values to enter : 2 (seperated by space) ==> 0.001 0.999
   B   P(B)
0  0  0.001
1  1  0.999

	Enter CPD values for

In [23]:
merged_cpds , merge_c = Merge_CPD(cpds) 

In [24]:
# Input Query
q_c = int(input('Enter the number of queries : '))
for _ in range(q_c):
  merge_c = merged_cpds.copy()
  inp = input('\nEnter query : ')
  print('\nInput Query : ',inp)
  print('Query Output : ',GetAnswer(inp,merge_c))

Enter the number of queries : 3

Enter query : A0 | E1

Input Query :  A0 | E1
Query Output :  0.8180929339980821

Enter query : C0 | A0, B1

Input Query :  C0 | A0, B1
Query Output :  0.00012034635681491333

Enter query : B1 A0 C0 D1 E1

Input Query :  B1 A0 C0 D1 E1
Query Output :  5.994e-08


#### Example 2. (Student Network In Daphne Koller)

In [ ]:
# Generate Network 
n,name_bay,card_bay,parent_bay,parent,total_node_parameters,cpds,rv_perm = GetModel()

Enter Network Data : 
Input number of variables : 5

Input information about variable  1
	Input Variable Name : d
	Input cardinality of variable : 2
	Enter the number of parents: 0

Input information about variable  2
	Input Variable Name : i
	Input cardinality of variable : 2
	Enter the number of parents: 0

Input information about variable  3
	Input Variable Name : g
	Input cardinality of variable : 3
	Enter the number of parents: 2
	Enter the parents(seperated by space) : i d

Input information about variable  4
	Input Variable Name : s
	Input cardinality of variable : 2
	Enter the number of parents: 1
	Enter the parents(seperated by space) : i

Input information about variable  5
	Input Variable Name : l
	Input cardinality of variable : 2
	Enter the number of parents: 1
	Enter the parents(seperated by space) : g

Input CPD data : 

	Enter CPD values for Node  d
	Total Values to enter : 2 (seperated by space) ==> 0.6 0.4
   d  P(d)
0  0   0.6
1  1   0.4

	Enter CPD values for Node  

In [ ]:
merged_cpds , merge_c = Merge_CPD(cpds) 

In [ ]:
# Input Query
q_c = int(input('Enter the number of queries : '))
for _ in range(q_c):
  merge_c = merged_cpds.copy()
  inp = input('\nEnter query : ')
  print('\nInput Query : ',inp)
  print('Query Output : ',GetAnswer(inp,merge_c))

Enter the number of queries : 5

Enter query : i1 d0 g1 s1 l0

Input Query :  i1 d0 g1 s1 l0
Query Output :  0.004608

Enter query : l1

Input Query :  l1
Query Output :  0.502336

Enter query : l1 | i0

Input Query :  l1 | i0
Query Output :  0.38859999999999983

Enter query : i1 | g2, d1

Input Query :  i1 | g2, d1
Query Output :  0.10909090909090911

Enter query : i1 | l0

Input Query :  i1 | l0
Query Output :  0.14002218364197533
